# Goal: showcase results of the V1 experiments with gradual causal ABA

The v1 experiment uses original causal aba framework but with additional rules.

In [ ]:
import pandas as pd
other_methods_data = pd.read_csv('../../results_pure_aba/stored_results_bnlearn_50rep_cpdag.csv')
v1_data = pd.read_csv('../../results/gradual/experiment_v1/cpdag_metrics.csv')
only_gradual_v1_data = pd.read_csv('../../results/gradual/stand_alone_gradual_v1/cpdag_metrics.csv')

In [ ]:
v1_data.head()

In [ ]:
DAG_ARCS_MAP = {'asia':8, 'cancer':4, 'earthquake':4, 'sachs':17, 'survey':6, 'alarm':46, 'child':25, 'insurance':52, 'hailfinder':66, 'hepar2':123}
DAG_NODES_MAP = {'asia':8, 'cancer':5, 'earthquake':5, 'sachs':11, 'survey':6, 'alarm':37, 'child':20, 'insurance':27, 'hailfinder':56, 'hepar2':70}

v1_data['model'] = 'f__' + v1_data['fact_ranking_method'] + '__r__' + v1_data['model_ranking_method']
only_gradual_v1_data['model'] = 'only_gradual__' + only_gradual_v1_data['model_ranking_method']

def process_v1_data(df):
    groupby_cols = ['dataset', 'n_nodes', 'model']
    df_grouped = df.groupby(groupby_cols, as_index=False).aggregate(
        sid_low_mean=('sid_low', 'mean'),
        sid_high_mean=('sid_high', 'mean'),
        sid_low_std=('sid_low', 'std'),
        sid_high_std=('sid_high', 'std'),
    )
    df_grouped['n_edges'] = df_grouped['dataset'].map(DAG_ARCS_MAP)
    df_grouped['p_SID_low_mean'] = df_grouped['sid_low_mean'] / df_grouped['n_edges']
    df_grouped['p_SID_high_mean'] = df_grouped['sid_high_mean'] / df_grouped['n_edges']
    df_grouped['p_SID_low_std'] = df_grouped['sid_low_std'] / df_grouped['n_edges']
    df_grouped['p_SID_high_std'] = df_grouped['sid_high_std'] / df_grouped['n_edges']
    return df_grouped

v1_grouped = process_v1_data(v1_data)
only_gradual_v1_grouped = process_v1_data(only_gradual_v1_data)

only_gradual_v1_grouped.head()

In [ ]:
other_methods_data.head()

In [ ]:
other_methods_data['n_edges'] = other_methods_data['dataset'].map(DAG_ARCS_MAP)
other_methods_data['n_nodes'] = other_methods_data['dataset'].map(DAG_NODES_MAP)
other_methods_data['p_SID_low_mean'] = other_methods_data['sid_low_mean'] / other_methods_data['n_edges']
other_methods_data['p_SID_high_mean'] = other_methods_data['sid_high_mean'] / other_methods_data['n_edges']
other_methods_data['p_SID_low_std'] = other_methods_data['sid_low_std'] / other_methods_data['n_edges']
other_methods_data['p_SID_high_std'] = other_methods_data['sid_high_std'] / other_methods_data['n_edges']
other_methods_data.head()

In [13]:
data_to_plot_columns = ['dataset', 'n_nodes', 'n_edges', 'model', 'p_SID_low_mean', 'p_SID_high_mean', 'p_SID_low_std', 'p_SID_high_std']

data_to_plot = pd.concat([other_methods_data[data_to_plot_columns], 
                         v1_grouped[data_to_plot_columns], 
                         only_gradual_v1_grouped[data_to_plot_columns]], ignore_index=True)

data_to_plot.head()

,dataset,n_nodes,n_edges,model,p_SID_low_mean,p_SID_high_mean,p_SID_low_std,p_SID_high_std
0,cancer,5,4,ABAPC (ASPforABA),2.435,2.66,0.3700,0.3850
1,cancer,5,4,Random,1.985,3.51,0.9075,0.7575
2,cancer,5,4,MPC,0.970,3.47,0.4525,0.4525
3,cancer,5,4,ABAPC (Ours),2.435,2.66,0.3700,0.3850
4,cancer,5,4,NOTEARS-MLP,2.360,2.56,0.3850,0.1075


In [14]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

def plot_sid_grouped_by_type(df: pd.DataFrame, model_order: list[str]):
    """
    Plot grouped bar chart showing Parent SID (Low/High) for each model, with spacing between groups.

    Args:
        df (pd.DataFrame): DataFrame for one dataset with columns:
            - dataset, n_nodes, n_edges, model
            - p_SID_low_mean, p_SID_high_mean
            - p_SID_low_std, p_SID_high_std
        model_order (list[str]): List of models in desired order
    """
    # Ensure only one dataset
    dataset_labels = df.apply(lambda row: f"{row['dataset']} |V|={row['n_nodes']}, |E|={row['n_edges']}", axis=1).unique()
    if len(dataset_labels) != 1:
        raise ValueError("Function only supports a single dataset.")
    dataset_label = dataset_labels[0]

    # Melt into long format
    df_low = df[["model", "p_SID_low_mean", "p_SID_low_std"]].copy()
    df_low["SID_type"] = "Low"
    df_low.rename(columns={"p_SID_low_mean": "SID_mean", "p_SID_low_std": "SID_std"}, inplace=True)

    df_high = df[["model", "p_SID_high_mean", "p_SID_high_std"]].copy()
    df_high["SID_type"] = "High"
    df_high.rename(columns={"p_SID_high_mean": "SID_mean", "p_SID_high_std": "SID_std"}, inplace=True)

    df_long = pd.concat([df_low, df_high], ignore_index=True)
    df_long = df_long[df_long["model"].isin(model_order)].copy()
    df_long["model"] = pd.Categorical(df_long["model"], categories=model_order, ordered=True)

    # Colors
    palette = px.colors.qualitative.Set2 + px.colors.qualitative.Plotly
    model_colors = {model: palette[i % len(palette)] for i, model in enumerate(model_order)}

    # Plotting positions
    n_models = len(model_order)
    group_spacing = n_models + 1  # space between Low and High
    x = []
    y = []
    errors = []
    colors = []
    model_names = []

    for sid_group_idx, sid_type in enumerate(["Low", "High"]):
        base = sid_group_idx * group_spacing
        for model_idx, model in enumerate(model_order):
            xpos = base + model_idx
            row = df_long[(df_long["model"] == model) & (df_long["SID_type"] == sid_type)]
            if not row.empty:
                x.append(xpos)
                y.append(row["SID_mean"].values[0])
                errors.append(row["SID_std"].values[0])
                colors.append(model_colors[model])
                model_names.append(model)

    # Main bars (no text labels)
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=x,
        y=y,
        error_y=dict(type='data', array=errors, visible=True),
        marker_color=colors,
        hovertemplate="Model: %{customdata}<br>SID: %{y:.2f}<extra></extra>",
        customdata=model_names,
        showlegend=False
    ))

    # Manual legend
    for model in model_order:
        fig.add_trace(go.Bar(
            x=[None], y=[None],
            marker_color=model_colors[model],
            name=model,
            showlegend=True
        ))

    # X-axis tick labels centered
    tick_positions = [
        (0 + (n_models - 1) / 2),
        (group_spacing + (n_models - 1) / 2)
    ]
    tick_labels = ["Low", "High"]

    fig.update_layout(
        title=f"Parent SID (Low vs High) — {dataset_label}",
        xaxis=dict(
            tickmode='array',
            tickvals=tick_positions,
            ticktext=tick_labels,
            title="SID Type"
        ),
        yaxis_title="Parent SID",
        bargap=0,
        showlegend=True,
        legend_title="Model",
    )

    fig.show()


In [15]:
data_to_plot.model.unique()

array(['ABAPC (ASPforABA)', 'Random', 'MPC', 'ABAPC (Ours)',
       'NOTEARS-MLP', 'f__original__r__arrows_mean',
       'f__original__r__arrows_sum', 'f__original__r__original',
       'f__original__r__refined_indep_facts',
       'f__refined_indep_facts__r__arrows_mean',
       'f__refined_indep_facts__r__arrows_sum',
       'f__refined_indep_facts__r__original',
       'f__refined_indep_facts__r__refined_indep_facts',
       'only_gradual__original_ranking', 'only_gradual__refined_ranking'],
      dtype=object)

In [11]:
model_order=['Random', 'NOTEARS-MLP', 'MPC',
            'only_gradual__original_ranking',
            'only_gradual__refined_ranking',
            'f__original__r__original',
            'f__original__r__refined_indep_facts',
            'f__original__r__arrows_sum', 
            'f__original__r__arrows_mean',
            'f__refined_indep_facts__r__original',
            'f__refined_indep_facts__r__refined_indep_facts',
            'f__refined_indep_facts__r__arrows_sum',
            'f__refined_indep_facts__r__arrows_mean']

In [12]:
plot_sid_grouped_by_type(data_to_plot[data_to_plot['dataset']=='cancer'],
                    model_order=model_order)

In [ ]:
plot_sid_grouped_by_type(data_to_plot[data_to_plot['dataset']=='earthquake'],
                    model_order=model_order)

In [ ]:
plot_sid_grouped_by_type(data_to_plot[data_to_plot['dataset']=='survey'],
                    model_order=model_order)

# Conclusion

No improvement observed when using refined independence facts. Likely because we missed out on dependence information.
However arrows mean and arrows sum ranking provides good results. This means that the arrow strengths do contain meaningful info.
This experiment is stepping stone for out standalone gradual-aba experiment.